In [1]:
# imports
import pandas as pd
import numpy as np
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# import the csv file
START_YR = 2010 # min is 1968
END_YR = 2020 # inclusive

def createDf(start_yr, end_yr):
    if (end_yr - start_yr) < 0:
        return (None, None)
    
    lenOfEndYr = None
    
    year_range = range(start_yr, end_yr + 1)
    url = 'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_%s.csv' % year_range[0]
    df = pd.read_csv(url).fillna(0)
    print("Downloading %s" % year_range[0])
    
    if (end_yr - start_yr) > 0: 
        for yr in year_range[1:]:
            url = 'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_%s.csv' % yr
            newDf = pd.read_csv(url).fillna(0)
            print("Downloading %s" % yr)
            df = df.append(newDf)
            if yr == end_yr:
                lenOfEndYr = newDf.shape[0]
    return (df, lenOfEndYr)

df, lenOfEndYr = createDf(START_YR, END_YR)
        
# df_2018 = pd.read_csv("./atp_matches_2018.csv").fillna(0)
# df_2019 = pd.read_csv("./atp_matches_2019.csv").fillna(0)
# df = df_2018.append(df_2019)
df['y'] = 1
print(df.size)
print(df.columns)

1504350
Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'y'],
      dtype='object')


In [4]:
# randomize order of player 1 and player 2 columns and y value
np.random.seed(1)

df['shouldSwap'] = np.random.randint(2, size = df.shape[0])
df.loc[df['shouldSwap'],['winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'loser_rank', 'loser_rank_points']] = df.loc[df['shouldSwap'],['loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'loser_rank', 'loser_rank_points', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'winner_rank', 'winner_rank_points']].values
df['y'] = df['shouldSwap'].apply(lambda x: int(not x))
# rename columns from winner to p1 and loser to p2
def renameWinnerLoser(x):
    if x == 'draw_size':
        return x
    return x.replace('winner_', 'p1_').replace('w_', 'p1_').replace('loser_', 'p2_').replace('l_', 'p2_')
df = df.rename(renameWinnerLoser, axis='columns')
def seedToInt(seed):
        resultStr = seed
        try:
            result = float(resultStr)
        except:
            result = sys.maxsize
        return result

df.loc[:, 'p1_seed'] = df['p1_seed'].apply(lambda x: seedToInt(x))
df.loc[:, 'p2_seed'] = df['p2_seed'].apply(lambda x: seedToInt(x))
df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,...,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p1_rank_points,p2_rank,p2_rank_points,y,shouldSwap
0,2010-339,Brisbane,Hard,32,A,20100103,1,103429,0.0,0,...,14.0,10.0,3.0,3.0,77.0,598.0,7.0,4410.0,0,1
1,2010-339,Brisbane,Hard,32,A,20100103,2,104999,0.0,0,...,19.0,10.0,0.0,0.0,78.0,590.0,134.0,400.0,0,1
2,2010-339,Brisbane,Hard,32,A,20100103,3,104755,0.0,0,...,14.0,14.0,7.0,11.0,52.0,850.0,88.0,568.0,1,0
3,2010-339,Brisbane,Hard,32,A,20100103,4,105051,0.0,Q,...,15.0,9.0,2.0,5.0,285.0,151.0,28.0,1260.0,1,0
4,2010-339,Brisbane,Hard,32,A,20100103,5,104607,4.0,0,...,14.0,9.0,6.0,9.0,20.0,1655.0,251.0,179.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,2020-M-DC-2020-WG2-PO-POL-HKG-01,Davis Cup WG2 PO: POL vs HKG,Hard,4,D,20200306,2,105668,0.0,0,...,0.0,0.0,0.0,0.0,461.0,68.0,960.0,11.0,1,0
743,2020-M-DC-2020-WG2-PO-POL-HKG-01,Davis Cup WG2 PO: POL vs HKG,Hard,4,D,20200306,4,209874,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
744,2020-M-DC-2020-WG2-PO-SYR-ZIM-01,Davis Cup WG2 PO: SYR vs ZIM,Hard,4,D,20200306,1,208518,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,813.0,18.0,1,0
745,2020-M-DC-2020-WG2-PO-SYR-ZIM-01,Davis Cup WG2 PO: SYR vs ZIM,Hard,4,D,20200306,2,111761,0.0,0,...,0.0,0.0,0.0,0.0,430.0,79.0,0.0,0.0,0,1


In [5]:
# feature engineering and data cleaning
def featEngModel1(dataset):
    def seedToInt(seed):
        resultStr = seed
        try:
            result = float(resultStr)
        except:
            result = sys.maxsize
        return result
    
    # add same winner and loser columns
    # newDataSet = dataset[['y', 'p1_rank', 'p1_rank_points', 'p1_ht', 'p1_age', 
    #                      'p2_rank', 'p2_rank_points', 'p2_ht', 'p2_age']]
    newDataSet = dataset[['y']]
    
    # add winner compute columns
    # newDataSet.loc[:, 'p1_seed'] = dataset['p1_seed'].apply(lambda x: seedToInt(x))
    # newDataSet.loc[:, 'p1_is_right_hand'] = dataset['p1_hand'].apply(lambda x: x == 'R')
    
    # add loser columns
    # newDataSet.loc[:, 'p2_seed'] = dataset['p2_seed'].apply(lambda x: seedToInt(x))
    # newDataSet.loc[:, 'p2_is_right_hand'] = dataset['p2_hand'].apply(lambda x: x == 'R')
    
    # add record between winner and loser
    pastRecord = dict()
    winRecordList = list()
    loseRecordList = list()
    for index, row in dataset.iterrows():
        match = (row['p1_id'], row['p2_id'])
        reverseMatch = (row['p2_id'], row['p1_id'])
        if match not in pastRecord:
            pastRecord[match] = 0
        if reverseMatch not in pastRecord:
            pastRecord[reverseMatch] = 0
        pastRecord[match] += 1
        winRecordList.append(pastRecord[match])
        loseRecordList.append(pastRecord[reverseMatch])
    newDataSet.loc[:, 'p1_past_wins'] = winRecordList
    newDataSet.loc[:, 'p1_past_losses'] = loseRecordList
    newDataSet.loc[:, 'rank_diff'] = dataset['p1_rank'].sub(dataset['p2_rank'], axis = 0)
    newDataSet.loc[:, 'seed_diff'] = dataset['p1_seed'].sub(dataset['p2_seed'], axis = 0)
    # newDataSet.loc[:, 'age_diff'] = dataset['p1_age'].sub(dataset['p2_age'], axis = 0)
    # newDataSet.loc[:, 'ht_diff'] = dataset['p1_ht'].sub(dataset['p2_ht'], axis = 0)
    
    # add tournament info columns
    # newDataSet.loc[:, 'is_4_draw'] = dataset['draw_size'].apply(lambda x: x == 4)
    # newDataSet.loc[:, 'is_8_draw'] = dataset['draw_size'].apply(lambda x: x == 8)
    # newDataSet.loc[:, 'is_32_draw'] = dataset['draw_size'].apply(lambda x: x == 32)
    # newDataSet.loc[:, 'is_64_draw'] = dataset['draw_size'].apply(lambda x: x == 64)
    # newDataSet.loc[:, 'is_hard'] = dataset['surface'].apply(lambda x: x == 'Hard')
    # newDataSet.loc[:, 'is_grass'] = dataset['surface'].apply(lambda x: x == 'Grass')
    # newDataSet.loc[:, 'is_A_level'] = dataset['tourney_level'].apply(lambda x: x == 'A')
    # newDataSet.loc[:, 'is_D_level'] = dataset['tourney_level'].apply(lambda x: x == 'D')
    # newDataSet.loc[:, 'is_F_level'] = dataset['tourney_level'].apply(lambda x: x == 'F')
    # newDataSet.loc[:, 'is_G_level'] = dataset['tourney_level'].apply(lambda x: x == 'G')
    
    print(newDataSet.columns)
    return newDataSet


# create datasets
def createTestTrainSplitModel1(dataset, start_yr, end_yr, lenOfEndYr): 
    newDataset = featEngModel1(dataset)
    msk = None
    if (end_yr - start_yr) <= 0:
        msk = np.arange(len(newDataset)) < (0.8 * newDataset.shape[0])
    else:
        msk = np.arange(len(newDataset)) < (newDataset.shape[0] - lenOfEndYr)
    train = newDataset[msk]
    test = newDataset[~msk]
    return (train, test)

modelTrain, modelTest = createTestTrainSplitModel1(df, START_YR, END_YR, lenOfEndYr)
print((modelTrain.shape[0], modelTest.shape[0]))

Index(['y', 'p1_past_wins', 'p1_past_losses', 'rank_diff', 'seed_diff'], dtype='object')
(29340, 747)


In [6]:
# Random (always win)
winsDf = modelTest['y'].value_counts()

# always win
winsDf['percent'] = winsDf.apply(lambda x: x / modelTest.shape[0])
winsDf

1                                                        382
0                                                        365
percent    1    0.511379
0    0.488621
Name: y, dtype: fl...
Name: y, dtype: object

In [7]:
# Better Rank
modelTestBetterRank = modelTest.copy()
modelTestBetterRank['is_p1_better_rank'] = modelTest['rank_diff'].apply(lambda x: int(x < 0))

# prediction
accuracyScoreBetterRank = accuracy_score(modelTest['y'], modelTestBetterRank['is_p1_better_rank'])
accuracyScoreBetterRank

0.4926372155287818

In [8]:
# Logistic Regression
model1Train = modelTrain
model1Test = modelTest

# train model
X = model1Train.loc[:, model1Train.columns != 'y']
y = model1Train['y']
clf_lr = LogisticRegression(random_state=0).fit(X, y)
print(clf_lr.coef_)

# test model quality
X_test = model1Test.loc[:, model1Test.columns != 'y']
y_test = model1Test['y']
clf_lr.predict(X_test)
clf_lr.score(X_test, y_test)

[[-1.52645832e-36 -5.37218792e-38 -1.29020614e-34  6.95895618e-20]]


0.4819277108433735

In [9]:
# Random Forest

# train model
clf_rf = RandomForestClassifier(max_depth=3, random_state=0)
clf_rf.fit(X, y)

# eval model
print(clf_rf.feature_importances_)
clf_rf.predict(X_test)
clf_rf.score(X_test, y_test)

[0.2013628  0.14256297 0.31938919 0.33668504]


0.47657295850066933

In [10]:
# SVM
# train model
clf_svm = svm.SVC()
clf_svm.fit(X, y)

#eval model
clf_svm.predict(X_test)
print(clf_svm.score(X_test, y_test))

0.4966532797858099
